## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np
from google.colab import drive
# import os
# checkpoint_path = "training_1/cp.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
drive.mount('/content/drive')

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("/content/drive/MyDrive/UPenn_Colab/Assignment/charity_data.csv")
application_df.head()

Mounted at /content/drive


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns={"EIN", "NAME"}, inplace = True)

In [3]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


## Variable Consideration

IS_SUCCESSFUL is the target while the rest of the columns are the feature.

## Preprocessing

In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()
application_lists = application_type_counts.index.to_list()
application_count = [[application_lists[i], application_type_counts[i]] for i in range(len(application_lists))]
application_type_df = pd.DataFrame(application_count)
filtered_application_type_df = application_type_df.loc[application_type_df[1] < 528]
application_types_to_replace = filtered_application_type_df[0]

# # Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# # Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'][application_df['CLASSIFICATION'].map(application_df['CLASSIFICATION'].value_counts()) > 1].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C2710        3
C0           3
C1267        2
C1256        2
C1234        2
C1246        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_counts = application_df['CLASSIFICATION'].value_counts()
classifications_lists = classifications_counts.index.to_list()
classifications_count = [[classifications_lists[i], classifications_counts[i]] for i in range(len(classifications_lists))]
classifications_df = pd.DataFrame(classifications_count)
filtered_classifications_df = classifications_df.loc[classifications_df[1] < 1883]
classifications_to_replace = filtered_classifications_df[0]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
askamt_min = application_df['ASK_AMT'].min()
askamt_max = 189280 * 6
application_df['ASK_AMT'].value_counts() 

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [11]:
# pd.cut(application_df['ASK_AMT'], 4).value_counts()
askamt_range = [5000, 193446, 381893, 570340, 758786]
application_df['ASK_AMT'] = application_df['ASK_AMT'].astype(str)

# Replace in dataframe
for key, amnt in enumerate(application_df['ASK_AMT']):
  if int(amnt) <= askamt_range[0]:
    application_df.at[key, 'ASK_AMT'] = '<=' + str(askamt_range[0])
  elif int(amnt) <= askamt_range[1]:
    application_df.at[key, 'ASK_AMT'] = '<=' + str(askamt_range[1])
  elif int(amnt) <= askamt_range[2]:
    application_df.at[key, 'ASK_AMT'] = '<=' + str(askamt_range[2])
  elif int(amnt) <= askamt_range[3]:
    application_df.at[key, 'ASK_AMT'] = '<=' + str(askamt_range[3])
  elif int(amnt) <= askamt_range[4]:
    application_df.at[key, 'ASK_AMT'] = '<=' + str(askamt_range[4])
  elif int(amnt) > askamt_range[4]:
    application_df.at[key, 'ASK_AMT'] = '>' + str(askamt_range[4])



application_df['ASK_AMT'].value_counts() 


<=5000      25398
<=193446     5447
>758786      1832
<=381893      919
<=570340      443
<=758786      260
Name: ASK_AMT, dtype: int64

In [12]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies = pd.get_dummies(application_df['APPLICATION_TYPE'])
classification_dummies = pd.get_dummies(application_df['CLASSIFICATION'])
askamt_dummies = pd.get_dummies(application_df['ASK_AMT'])
affiliation_dummies = pd.get_dummies(application_df['AFFILIATION'])
usecase_dummies = pd.get_dummies(application_df['USE_CASE'])
organization_dummies = pd.get_dummies(application_df['ORGANIZATION'])
incomeamt_dummies = pd.get_dummies(application_df['INCOME_AMT'])

In [13]:
application_dummies = application_dummies.iloc[: , :-1]
classification_dummies = classification_dummies.iloc[: , :-1]
askamt_dummies = askamt_dummies.iloc[: , :-1]
affiliation_dummies = affiliation_dummies.iloc[: , :-1]
usecase_dummies = usecase_dummies.iloc[: , :-1]
organization_dummies = organization_dummies.iloc[: , :-1]
incomeamt_dummies = incomeamt_dummies.iloc[: , :-1]

In [14]:
filtered_df = application_df.drop(columns={'APPLICATION_TYPE', 'CLASSIFICATION', 'ASK_AMT', 'AFFILIATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'})

In [15]:
final_df = pd.concat([filtered_df, application_dummies, classification_dummies, askamt_dummies, affiliation_dummies, usecase_dummies, organization_dummies], axis=1, join='inner')

In [16]:
final_df

,STATUS,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,Other,T10,T19,T3,T4,T5,T6,...,Independent,National,Other,CommunityServ,Heathcare,Other,Preservation,Association,Co-operative,Corporation
0,1,N,1,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1,N,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,1,0,1,0
2,1,N,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,1,N,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,1,N,1,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,N,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
34295,1,N,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
34296,1,N,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0
34297,1,N,1,0,0,0,0,0,1,0,...,1,0,0,0,0,0,0,1,0,0


In [17]:
final_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [18]:
for key, value in enumerate(final_df['SPECIAL_CONSIDERATIONS']):
  if amnt == 'Y':
    final_df.at[key, 'SPECIAL_CONSIDERATIONS'] = 1
  else:
    final_df.at[key, 'SPECIAL_CONSIDERATIONS'] = 0

In [19]:
# Split our preprocessed data into our features and target arrays
X, y = final_df.drop(columns={'IS_SUCCESSFUL'}), final_df['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [20]:
X.columns

Index(['STATUS', 'SPECIAL_CONSIDERATIONS', 'Other', 'T10', 'T19', 'T3', 'T4',
       'T5', 'T6', 'T7', 'C1000', 'C1200', 'C2000', 'C2100', 'C3000',
       '<=193446', '<=381893', '<=5000', '<=570340', '<=758786',
       'CompanySponsored', 'Family/Parent', 'Independent', 'National', 'Other',
       'CommunityServ', 'Heathcare', 'Other', 'Preservation', 'Association',
       'Co-operative', 'Corporation'],
      dtype='object')

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model (#1)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                330       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 1)                 9         
                                                                 
Total params: 427
Trainable params: 427
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [24]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6188 - accuracy: 0.6805
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5723 - accuracy: 0.7195
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5643 - accuracy: 0.7226
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5609 - accuracy: 0.7263
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5588 - accuracy: 0.7266
Epoch 6/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5575 - accuracy: 0.7267
Epoch 7/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5565 - accuracy: 0.7276
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5554 - accuracy: 0.7284
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5547 - accuracy: 0.7285
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5540 - accura

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5548 - accuracy: 0.7269 - 441ms/epoch - 2ms/step
Loss: 0.5548254251480103, Accuracy: 0.7268804907798767


In [26]:
# from keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint("AlphabetSoup.h5", monitor='loss', verbose=1,
#     save_best_only=True, mode='auto', period=5)
nn.save("/content/drive/MyDrive/UPenn_Colab/Assignment/AlphabetSoupCharity.h5")


## Compile, Train and Evaluate the Model (#2)

In [27]:
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 30
hidden_nodes_layer2 = 7
hidden_nodes_layer3 = 2

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 30)                990       
                                                                 
 dense_4 (Dense)             (None, 7)                 217       
                                                                 
 dense_5 (Dense)             (None, 2)                 16        
                                                                 
 dense_6 (Dense)             (None, 1)                 3         
                                                                 
Total params: 1,226
Trainable params: 1,226
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6726 - accuracy: 0.6127
Epoch 2/100
804/804 [==============================] - 1s 2ms/step - loss: 0.6120 - accuracy: 0.7209
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5902 - accuracy: 0.7217
Epoch 4/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5769 - accuracy: 0.7241
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5669 - accuracy: 0.7286
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5621 - accuracy: 0.7293
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5594 - accuracy: 0.7300
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5578 - accuracy: 0.7314
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5565 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5557 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5545 - accuracy: 0.7289 - 600ms/epoch - 2ms/step
Loss: 0.5544982552528381, Accuracy: 0.728863000869751


In [31]:
nn.save("/content/drive/MyDrive/UPenn_Colab/Assignment/AlphabetSoupCharity_Optimization.h5") #best accuracy score

## Compile, Train and Evaluate the Model (#3)

In [32]:
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 70
hidden_nodes_layer2 = 21
hidden_nodes_layer3 = 7

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_7 (Dense)             (None, 70)                2310      
                                                                 
 dense_8 (Dense)             (None, 21)                1491      
                                                                 
 dense_9 (Dense)             (None, 7)                 154       
                                                                 
 dense_10 (Dense)            (None, 1)                 8         
                                                                 
Total params: 3,963
Trainable params: 3,963
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.6028 - accuracy: 0.7013
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5688 - accuracy: 0.7271
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5616 - accuracy: 0.7270
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5579 - accuracy: 0.7276
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7278
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7296
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5527 - accuracy: 0.7300
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5516 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5507 - accuracy: 0.7308
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5496 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
# from keras.callbacks import ModelCheckpoint

# checkpoint = ModelCheckpoint("AlphabetSoup_Optimization.h5", monitor='loss', verbose=1,
#     save_best_only=True, mode='auto', period=5)
nn.save("AlphabetSoupCharity_Optimization.h5")

# Report
## Overview
The purpose of this analysis was to create a classifier that would be able to predict whether or not applicants would be successful if funded by Alphabet Soup. The classifier is a neural network with the option of tuning hyperparameters to optimize how well the classifier will perform. This report will give a detailed analysis of the preprocessing of the data and how the hyperparameters were tuned along with a recommendation of another potential model for this classification problem.

## Results
### Data Preprocessing
The features we considered for our model are:
1.   Application Type
2.   Affiliation
3.   Government Classification
4.   Use Case
5.   Organization
6.   Active Status
7.   Income
8.   Speccial Considerations
9.   Funding Amount Requested

The variable that should not be included to train our model is the identification data.

The target variable we wanted to consider for our classification of our model was whether or not the organization was successful

### Compiling, Training, and Evaluating the Model
I have performed more tests than displayed in my analysis. For my best model I used 39 total neurons, 3 total hidden layers, and a combination of relu and sigmoid activation functions at 100 epochs. I was not able to achieve the target model performance.
The four hyperparameters I played around with were:

*   Amount of neurons
*   Amonut of hidden layers
*   Activation function
*   Number of Epochs


The values I for each hyper parameter:

Range of neurons: 2-70, where 70 is the highest and I would decrease neurons for every next layer

Activation Function: Input: Relu, Output:Sigmoid, Hidden Layers: Combination of Relu/Sigmoid

Amount of hidden Layers: 1-3, with a total of 4 layers, including the output.

Nuumber of Epochs: I used primarily 100 and 200 epochs, but 100 seemed to perform better with my set of tuned hyperparameters.

## Summary
Overall, I was not able to achieve an accuracy of above 75%. There are some ways to optimize the model, most of which would be trial and error. There is a keras tuner package that would also help optimize the hyperparameters. With the labels of the data, we should be able to run a supervised ML model for this classification problem. One I would potentially recommend would be the Random Forest Classifier. It has a similar approach with the neural network in that it has multiple layers, which would be unique to a set of features and then come down to some classification.




